In [155]:
from edgar import *

# Tell the SEC who you are
set_identity("senorpepito87@gmail.com")


[15:08:07] INFO     Identity of the Edgar REST client set to [senorpepito87@gmail.com]                  ]8;id=138811;file:///Users/adityakunte/Desktop/quant_strat/venv/lib/python3.10/site-packages/edgar/core.py\core.py]8;;\:]8;id=219386;file:///Users/adityakunte/Desktop/quant_strat/venv/lib/python3.10/site-packages/edgar/core.py#158\158]8;;\

In [88]:
# get a list of all stocks with their CIKs
import pandas as pd
cik_df = pd.read_csv("ticker_to_cik_processed.csv")
stocks = pd.read_csv("daily_stock_prices.csv")
stocks['Net Income'] = 0 
stocks['SE'] = 0
# stocks.loc[stocks['Symbol'] == 'AAPL', 'Net Income'] = 50000000000  # Example value



In [ ]:
net_income_terms = [
    "Net Income", "Net Profit", "Net Earnings", "Profit After Tax", 
    "Net Revenue", "Bottom Line", "Income After Tax", 
    "Net Profit After Tax", "Net Income (Loss)", "Net Gain",
    "Profit for the Year", "Net Profit for the Year",
    "Earnings After Tax", "Net Income (from Continuing Operations)"
]

# Flag to start processing only after encountering "COIN"
start_processing = False

for value in stocks['Symbol']:

    # Check if we should start processing
    if not start_processing:
        if value != "USFD":
            continue
        else:
            start_processing = True

    result = cik_df.loc[cik_df['Symbol'] == value.lower(), 'CIK']
    cik = str(result.iloc[0]).zfill(10) 
    company = Company(cik)
    if company is None:
        print(f"{value}: does not exist in edgartools")
        continue
    filings = company.get_filings(form="10-K")
    if filings is None:
        print(f"{value}: company filings not found")
        continue
    latest = filings.latest(1)
    if latest is None:
        print(f"{value}: could not find latest 10-k filing")
        continue
    
    tenk = latest.obj()
    if tenk is None:
        print(f"{value}: could not find latest 10-k filing")
        continue
    
    bs = tenk.financials.get_balance_sheet()
    if bs is None:
        print(f"{value}: balance sheet not found")
        continue
    bal_sheet = bs.get_dataframe()
    if bal_sheet is None:
        print(f"{value}: balance sheet not found")
        continue

    inc_s = tenk.financials.get_income_statement()
    if inc_s is None:
        print(f"{value}: income statement not found")
        continue
    inc_statement = inc_s.get_dataframe()
    if inc_statement is None:
        print(f"{value}: income statement not found")
        continue
    # Calculate shareholders' equity
    asset_terms = ["total assets"]
    liability_terms = ["total liabilities"]

    pattern = '|'.join(asset_terms)
    pattern2 = '|'.join(liability_terms)
    if '2023' not in bal_sheet.columns or '2023' not in inc_statement.columns:
        print(f"{value}: 2023 column doesn't exist")
        continue
    filtered_assets_rows = bal_sheet[bal_sheet.index.str.contains(pattern, case=False, regex=True)]['2023']
    filtered_liabilities_rows = bal_sheet[bal_sheet.index.str.contains(pattern2, case=False, regex=True)]['2023']

    if filtered_assets_rows.empty:
        print(f"{value}: assets don't exist")
        continue

    if filtered_liabilities_rows.empty:
        print(f"{value}: liabilities don't exist")
        continue

    total_assets = int(float(filtered_assets_rows.iloc[0]))
    total_liabilities = int(float(filtered_liabilities_rows.iloc[0]))

    shareholder_equity = total_assets - total_liabilities
    stocks.loc[stocks['Symbol'] == value, 'SE'] = shareholder_equity 

    pattern = '|'.join(net_income_terms)
    filtered_net_income_terms = inc_statement[inc_statement.index.str.contains(pattern, case=False, regex=True)]['2023']

    if filtered_net_income_terms.empty:
        print(f"{value}: could not find net income")
        continue

    net_income = filtered_net_income_terms.iloc[0]
    if net_income is not None:
        if net_income == '':
            print(f"{value}: could not find net income")
        else:
            stocks.loc[stocks['Symbol'] == value, 'Net Income'] = int(float(net_income))
    else:
        print(f"{value}: could not find net income")


In [165]:
import pandas as pd

# Load the data
data = pd.read_csv('final.csv')

# Filter the data where 'Net Income' or 'SE' columns are either empty, 0, or NaN
filtered_symbols = data[
    (data['Net Income'].isna()) | (data['SE'].isna()) | 
    (data['Net Income'] == 0) | (data['SE'] == 0) |
    (data['Net Income'] == '') | (data['SE'] == '')
]['Symbol'].unique().tolist()

# Display the list of symbols
len(filtered_symbols)


131

In [166]:
import pandas as pd

# Load the data
data = pd.read_csv('final.csv')

# Drop rows where 'Net Income' or 'SE' columns are either empty, 0, or NaN
cleaned_data = data.dropna(subset=['Net Income', 'SE'])
cleaned_data = cleaned_data[
    (cleaned_data['Net Income'] != 0) & (cleaned_data['SE'] != 0) &
    (cleaned_data['Net Income'] != '') & (cleaned_data['SE'] != '')
]

cleaned_data

,Unnamed: 0,Symbol,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,...,2023-12-24,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31,SE,Net Income
2,2,WFC,NaN,NaN,39.705818,40.522934,40.304398,40.665447,NaN,NaN,...,NaN,NaN,48.640415,48.336658,48.483635,48.228874,NaN,NaN,1.874430e+11,4.368000e+09
3,3,QCOM,NaN,NaN,103.454712,107.633430,105.577858,111.310318,NaN,NaN,...,NaN,NaN,144.134201,144.391846,144.530548,143.311768,NaN,NaN,2.158100e+10,7.232000e+09
4,4,TXN,NaN,NaN,154.873688,160.529221,158.403641,166.213303,NaN,NaN,...,NaN,NaN,167.171631,167.582672,168.062241,166.829086,NaN,NaN,1.689700e+10,6.510000e+09
5,5,GE,NaN,NaN,52.601925,55.663586,56.527878,57.043285,NaN,NaN,...,NaN,NaN,101.315971,101.960182,102.055618,101.506844,NaN,NaN,2.857900e+10,9.443000e+09
6,6,IBM,NaN,NaN,131.173401,132.146454,130.765640,133.165802,NaN,NaN,...,NaN,NaN,158.757217,159.000397,159.282486,159.087936,NaN,NaN,2.261300e+10,7.502000e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,335,SKX,NaN,NaN,41.830002,43.450001,43.220001,44.590000,NaN,NaN,...,NaN,NaN,62.669998,62.660000,62.560001,62.340000,NaN,NaN,4.400042e+09,6.499230e+08
338,338,CNM,NaN,NaN,19.540001,20.260000,20.070000,20.760000,NaN,NaN,...,NaN,NaN,40.040001,40.410000,40.599998,40.410000,NaN,NaN,2.410000e+09,5.810000e+08
339,339,SFM,NaN,NaN,30.920000,30.940001,30.680000,31.709999,NaN,NaN,...,NaN,NaN,47.930000,48.080002,47.810001,48.110001,NaN,NaN,1.148547e+09,2.588560e+08
340,340,UNM,NaN,NaN,38.132809,38.454922,38.739143,38.975994,NaN,NaN,...,NaN,NaN,44.086502,44.164703,44.272232,44.203804,NaN,NaN,9.651400e+09,1.283800e+09


In [167]:
# Save the cleaned data to a new CSV file
cleaned_data['ROE'] = cleaned_data['Net Income'] / cleaned_data['SE']
cleaned_data

,Unnamed: 0,Symbol,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,...,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31,SE,Net Income,ROE
2,2,WFC,NaN,NaN,39.705818,40.522934,40.304398,40.665447,NaN,NaN,...,NaN,48.640415,48.336658,48.483635,48.228874,NaN,NaN,1.874430e+11,4.368000e+09,0.023303
3,3,QCOM,NaN,NaN,103.454712,107.633430,105.577858,111.310318,NaN,NaN,...,NaN,144.134201,144.391846,144.530548,143.311768,NaN,NaN,2.158100e+10,7.232000e+09,0.335110
4,4,TXN,NaN,NaN,154.873688,160.529221,158.403641,166.213303,NaN,NaN,...,NaN,167.171631,167.582672,168.062241,166.829086,NaN,NaN,1.689700e+10,6.510000e+09,0.385275
5,5,GE,NaN,NaN,52.601925,55.663586,56.527878,57.043285,NaN,NaN,...,NaN,101.315971,101.960182,102.055618,101.506844,NaN,NaN,2.857900e+10,9.443000e+09,0.330417
6,6,IBM,NaN,NaN,131.173401,132.146454,130.765640,133.165802,NaN,NaN,...,NaN,158.757217,159.000397,159.282486,159.087936,NaN,NaN,2.261300e+10,7.502000e+09,0.331756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,335,SKX,NaN,NaN,41.830002,43.450001,43.220001,44.590000,NaN,NaN,...,NaN,62.669998,62.660000,62.560001,62.340000,NaN,NaN,4.400042e+09,6.499230e+08,0.147708
338,338,CNM,NaN,NaN,19.540001,20.260000,20.070000,20.760000,NaN,NaN,...,NaN,40.040001,40.410000,40.599998,40.410000,NaN,NaN,2.410000e+09,5.810000e+08,0.241079
339,339,SFM,NaN,NaN,30.920000,30.940001,30.680000,31.709999,NaN,NaN,...,NaN,47.930000,48.080002,47.810001,48.110001,NaN,NaN,1.148547e+09,2.588560e+08,0.225377
340,340,UNM,NaN,NaN,38.132809,38.454922,38.739143,38.975994,NaN,NaN,...,NaN,44.086502,44.164703,44.272232,44.203804,NaN,NaN,9.651400e+09,1.283800e+09,0.133017


In [168]:
cleaned_data.to_csv('cleaned_data.csv', index=False)

,Unnamed: 0,Symbol,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,...,2023-12-25,2023-12-26,2023-12-27,2023-12-28,2023-12-29,2023-12-30,2023-12-31,SE,Net Income,ROE
3,3,QCOM,NaN,NaN,103.454712,107.633430,105.577858,111.310318,NaN,NaN,...,NaN,144.134201,144.391846,144.530548,143.311768,NaN,NaN,2.158100e+10,7.232000e+09,0.335110
4,4,TXN,NaN,NaN,154.873688,160.529221,158.403641,166.213303,NaN,NaN,...,NaN,167.171631,167.582672,168.062241,166.829086,NaN,NaN,1.689700e+10,6.510000e+09,0.385275
5,5,GE,NaN,NaN,52.601925,55.663586,56.527878,57.043285,NaN,NaN,...,NaN,101.315971,101.960182,102.055618,101.506844,NaN,NaN,2.857900e+10,9.443000e+09,0.330417
6,6,IBM,NaN,NaN,131.173401,132.146454,130.765640,133.165802,NaN,NaN,...,NaN,158.757217,159.000397,159.282486,159.087936,NaN,NaN,2.261300e+10,7.502000e+09,0.331756
7,7,AXP,NaN,NaN,143.634781,146.973770,143.456665,147.120728,NaN,NaN,...,NaN,184.239868,185.329681,186.062836,185.607101,NaN,NaN,2.794900e+10,8.374000e+09,0.299617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,335,SKX,NaN,NaN,41.830002,43.450001,43.220001,44.590000,NaN,NaN,...,NaN,62.669998,62.660000,62.560001,62.340000,NaN,NaN,4.400042e+09,6.499230e+08,0.147708
338,338,CNM,NaN,NaN,19.540001,20.260000,20.070000,20.760000,NaN,NaN,...,NaN,40.040001,40.410000,40.599998,40.410000,NaN,NaN,2.410000e+09,5.810000e+08,0.241079
339,339,SFM,NaN,NaN,30.920000,30.940001,30.680000,31.709999,NaN,NaN,...,NaN,47.930000,48.080002,47.810001,48.110001,NaN,NaN,1.148547e+09,2.588560e+08,0.225377
340,340,UNM,NaN,NaN,38.132809,38.454922,38.739143,38.975994,NaN,NaN,...,NaN,44.086502,44.164703,44.272232,44.203804,NaN,NaN,9.651400e+09,1.283800e+09,0.133017
